In [147]:
import TSeriesPreproccesing as TSPP
import SequencePreprocessing as SP
import ClusterProcessing as CP
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
import utils as utils
from tslearn.clustering import TimeSeriesKMeans
import os 
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [148]:
reload(TSPP)
reload(SP)
reload(CP)
pd.set_option('display.max_columns', 100)

In [149]:
tickers = ['aapl','nvda','amd','msft']
start = '2022-01-01'
target_cols = ['sumPctChgclose_1','sumPctChgclose_2','sumPctChgclose_3','sumPctChgclose_4','sumPctChgclose_5','sumPctChgclose_6']
n_steps = 100
interval = '1h'
group_params = CP.StockClusterGroupParams(start_date = start, tickers = tickers, interval = interval, target_cols = target_cols, n_steps = n_steps)

In [150]:

stockDataSet = TSPP.StockDataSet(group_params=group_params)

stockDataSet.preprocess_pipeline()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [151]:
stockSequence = stockDataSet.create_sequence_set()

In [155]:

stockSequence.create_combined_sequence()
print(len(stockSequence.group_params.test_seq_elements))
# print(stockSequence.group_params.test_seq_elements[0].seq_x.shape)
stockSequence.scale_sequences()
print(len(stockSequence.group_params.test_seq_elements))



4


In [153]:
# # test_features = x_sbsg_feature_sets[0].cols
# test_features = ['close','sma10','ema20','ema50']
# test_indices = [stockSequence.X_feature_dict[feature] for feature in test_features]

# fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
# x_values = np.arange(len(stockSequence.X_train[50,:,20]))
# for i in test_indices:
#     axes[0].plot(x_values,stockSequence.X_train[75,:,i])
#     axes[1].plot(x_values,stockSequence.X_train_scaled[75,:,i])

In [154]:
X_train, y_train = SP.SequenceElement.create_array(stockSequence.group_params.train_seq_elements,scaled = True)
X_test, y_test = SP.SequenceElement.create_array(stockSequence.group_params.test_seq_elements,scaled = True)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4, 100, 49)
(4, 6)
(4, 100, 49)
(4, 6)


In [57]:
# cluster_features = ['sumPctChgclose_1','sumPctChgclose_3','sumPctChgclose_6']
cluster_features = ['close','ema100']
cluster_indices = [stockSequence.group_params.X_feature_dict[feature] for feature in cluster_features]
X_cluster = utils.create_Cluster_Seq(X_train,cluster_indices)

In [58]:
import math
n_clusters = math.ceil(math.sqrt(len(X_cluster))) // 3
# A good rule of thumb is choosing k as the square root of the number of points in the training data set in kNN

km = TimeSeriesKMeans(n_clusters=n_clusters)

labels = km.fit_predict(X_cluster)

In [59]:
reload(utils)
threshold_factor = 1 # Adjust as necessary

X_cluster_adj, X_train_adj, y_train_adj, labels_adj = utils.remove_outliers(X_cluster,X_train,y_train,labels,km,threshold_factor)
display(X_cluster.shape)
display(X_train.shape)
display(y_train.shape)
display(X_cluster_adj.shape)
display(X_train_adj.shape)
display(y_train_adj.shape)
display(labels_adj.shape)

(1012, 100, 2)

(1012, 100, 49)

(1012, 6)

(870, 100, 2)

(870, 100, 49)

(870, 6)

(870,)

In [60]:
reload(utils)
utils.visualizeData(labels_adj,X_cluster_adj)


In [12]:
from keras.layers import RepeatVector, TimeDistributed

def create_model():
    # Encoder
    model_lstm = Sequential()
    
    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True, input_shape=(None, len(X_train_adj[0][0]))))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=100, activation='tanh',return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=100, activation='tanh'))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))
    
    # Repeat the encoder output (which is the last hidden state) 
    # for 'n' times where 'n' is the number of prediction steps
    model_lstm.add(RepeatVector(6))  # Assuming you are predicting for 6 steps

    # Decoder
    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))

    model_lstm.add(LSTM(units=250, activation='tanh', return_sequences=True))
    model_lstm.add(BatchNormalization())
    model_lstm.add(Dropout(0.2))
    
    model_lstm.add(TimeDistributed(Dense(1)))  # Predict one value for each time step

    optimizer = Adam(learning_rate=0.001)
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=1e-5)

    model_lstm.compile(optimizer=optimizer, loss="mse")
    return model_lstm

In [13]:
# display(X_train[0:1,:,-1])
X_cluster_test = utils.create_Cluster_Seq(X_test,cluster_indices)
# display(X_cluster_test.shape)

cluster_assignments = km.predict(X_cluster_test)


for lab in set(labels_adj): 
    filtered_x_train, filtered_y_train = utils.filter_clusters(labels_adj, lab, X_train_adj, y_train_adj)


    # values, counts = np.unique(cluster_assignments, return_counts=True)

    # # Display the results
    # for value, count in zip(values, counts):
    #     print(f"Value: {value}, Count: {count}")

    X_cluster_test_adj,X_test_adj, y_test_adj, labels_test_adj = utils.remove_outliers(X_cluster_test,X_test,y_test,cluster_assignments,km,threshold_factor)


    filtered_x_test, filtered_y_test = utils.filter_clusters(labels_test_adj, lab, X_test_adj, y_test_adj)
    
    # display(X_cluster_test_adj.shape)
    # display(filtered_x_test.shape)
    # display(filtered_y_test.shape)

    if len(filtered_x_test) == 0:
        continue

    # display(filtered_x_train.shape)
    model_lstm = create_model()
    print(filtered_x_test.shape)
    print(filtered_y_test.shape)

    y_train_reshaped = np.expand_dims(y_train, axis=-1)
    y_test_reshaped = np.expand_dims(y_test, axis=-1)
    
    model_lstm.fit(filtered_x_train, filtered_y_train, batch_size=30, epochs=300, verbose=1)

    
    
    predicted_stock_price = model_lstm.predict(filtered_x_test)
    predicted_stock_price = np.squeeze(predicted_stock_price, axis=-1)
    print(predicted_stock_price.shape)



    # predicted_stock_price = predictiveScaler.inverse_transform(predicted_stock_price)
    # filtered_y_test = predictiveScaler.inverse_transform(filtered_y_test)

    # Assuming predicted_stock_price has two columns
    results = pd.DataFrame({
        'predicted_1': predicted_stock_price[:, 0],
        'predicted_2': predicted_stock_price[:, 1],
        'predicted_3': predicted_stock_price[:, 2],
        'predicted_4': predicted_stock_price[:, 3],
        'predicted_5': predicted_stock_price[:, 4],
        'predicted_6': predicted_stock_price[:, 5],
        'real_1': filtered_y_test[:, 0],
        'real_2': filtered_y_test[:, 1],
        'real_3': filtered_y_test[:, 2],
        'real_4': filtered_y_test[:, 3],
        'real_5': filtered_y_test[:, 4],
        'real_6': filtered_y_test[:, 5]
    })

    # Assuming a correct prediction means both predicted features have the same sign as their corresponding real values
    results['same_1'] = ((results['predicted_1'] > 0) & (results['real_1'] > 0)) | ((results['predicted_1'] < 0) & (results['real_1'] < 0))
    results['same_2'] = ((results['predicted_2'] > 0) & (results['real_2'] > 0)) | ((results['predicted_2'] < 0) & (results['real_2'] < 0))
    results['same_3'] = ((results['predicted_3'] > 0) & (results['real_3'] > 0)) | ((results['predicted_3'] < 0) & (results['real_3'] < 0))
    results['same_4'] = ((results['predicted_4'] > 0) & (results['real_4'] > 0)) | ((results['predicted_4'] < 0) & (results['real_4'] < 0))
    results['same_5'] = ((results['predicted_5'] > 0) & (results['real_5'] > 0)) | ((results['predicted_5'] < 0) & (results['real_5'] < 0))
    results['same_6'] = ((results['predicted_6'] > 0) & (results['real_6'] > 0)) | ((results['predicted_6'] < 0) & (results['real_6'] < 0))
    accuracy1 = results['same_1'].sum() / len(results) * 100
    accuracy2 = results['same_2'].sum() / len(results) * 100
    accuracy3 = results['same_3'].sum() / len(results) * 100
    accuracy4 = results['same_4'].sum() / len(results) * 100
    accuracy5 = results['same_5'].sum() / len(results) * 100
    accuracy6 = results['same_6'].sum() / len(results) * 100

    # Construct the string
    output_string = (
        "Cluster Number: " + str(lab) +
        " \nAccuracy1D " + str(accuracy1) + " PredictedRet: " + str(results['predicted_1'].mean()) + " ActRet " + str(results['real_1'].mean() ) +
        " \nAccuracy2D " + str(accuracy2) + " PredictedRet: " + str(results['predicted_2'].mean()) + " ActRet " + str(results['real_2'].mean() ) +
        " \nAccuracy3D " + str(accuracy3) + " PredictedRet: " + str(results['predicted_3'].mean()) + " ActRet " + str(results['real_3'].mean() ) +
        " \nAccuracy4D " + str(accuracy4) + " PredictedRet: " + str(results['predicted_4'].mean()) + " ActRet " + str(results['real_4'].mean() ) +
        " \nAccuracy5D " + str(accuracy5) + " PredictedRet: " + str(results['predicted_5'].mean()) + " ActRet " + str(results['real_5'].mean() ) +
        " \nAccuracy6D " + str(accuracy6) + " PredictedRet: " + str(results['predicted_6'].mean()) + " ActRet " + str(results['real_6'].mean() ) +
        " Test set length: " + str(len(filtered_y_test)) + "\n"
    )

    # Write the string to a file
    with open('fuck.txt', 'a') as f:
        f.write(output_string)






2023-10-14 16:26:18.067412: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-14 16:26:18.067433: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: acet116-lnx-13
2023-10-14 16:26:18.067438: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: acet116-lnx-13
2023-10-14 16:26:18.067496: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.98.0
2023-10-14 16:26:18.067509: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.98.0
2023-10-14 16:26:18.067513: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.98.0
2023-10-14 16:26:18.524192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor 

(69, 20, 49)
(69, 6)
Epoch 1/300


2023-10-14 16:26:19.572094: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-14 16:26:19.573282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-14 16:26:19.574021: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2023-10-14 16:26:19.832282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-14 16:26:19.833373: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-14 16:26:19.834156: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

7/7 [==============================] - 7s 50ms/step - loss: 2.5896
Epoch 2/300
7/7 [==============================] - 0s 49ms/step - loss: 1.7443
Epoch 3/300
7/7 [==============================] - 0s 49ms/step - loss: 1.1458
Epoch 4/300
7/7 [==============================] - 0s 49ms/step - loss: 1.0614
Epoch 5/300
7/7 [==============================] - 0s 49ms/step - loss: 0.8899
Epoch 6/300
7/7 [==============================] - 0s 50ms/step - loss: 0.8355
Epoch 7/300
7/7 [==============================] - 0s 50ms/step - loss: 0.8565
Epoch 8/300
7/7 [==============================] - 0s 49ms/step - loss: 0.7917
Epoch 9/300
7/7 [==============================] - 0s 49ms/step - loss: 0.6960
Epoch 10/300
7/7 [==============================] - 0s 50ms/step - loss: 0.6401
Epoch 11/300
7/7 [==============================] - 0s 49ms/step - loss: 0.6490
Epoch 12/300
7/7 [==============================] - 0s 50ms/step - loss: 0.5750
Epoch 13/300
7/7 [==============================] - 0s 49ms/s

...

(46, 20, 49)
(46, 6)
Epoch 1/300


2023-10-14 17:37:27.882107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-14 17:37:27.883186: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-14 17:37:27.883945: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

8/8 [==============================] - 7s 47ms/step - loss: 2.7132
Epoch 2/300
8/8 [==============================] - 0s 46ms/step - loss: 1.5523
Epoch 3/300
8/8 [==============================] - 0s 47ms/step - loss: 1.5696
Epoch 4/300
8/8 [==============================] - 0s 47ms/step - loss: 1.0854
Epoch 5/300
8/8 [==============================] - 0s 46ms/step - loss: 1.1342
Epoch 6/300
8/8 [==============================] - 0s 46ms/step - loss: 1.0239
Epoch 7/300
8/8 [==============================] - 0s 46ms/step - loss: 0.7638
Epoch 8/300
8/8 [==============================] - 0s 46ms/step - loss: 0.8005
Epoch 9/300
8/8 [==============================] - 0s 46ms/step - loss: 0.6940
Epoch 10/300
8/8 [==============================] - 0s 47ms/step - loss: 0.6476
Epoch 11/300
8/8 [==============================] - 0s 50ms/step - loss: 0.5820
Epoch 12/300
8/8 [==============================] - 0s 48ms/step - loss: 0.7445
Epoch 13/300
8/8 [==============================] - 0s 46ms/s

2023-10-14 17:39:22.136167: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-10-14 17:39:22.137263: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-10-14 17:39:22.137987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2/2 [==============================] - 1s 10ms/step
(46, 6)
